In [26]:
import logging

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

from webapp import create_app
from webapp.model import Cluster, ClusterPoint, db, Point

In [27]:
MIN_DISTANCE = 5  # [km]
MAX_DISTANCE = 5  # [km]
DISTANCE_STEP = 5  # [km]
KMS_PER_RADIAN = 6371.0088  # [km]
distance_list = [r for r in range(MIN_DISTANCE, MAX_DISTANCE + DISTANCE_STEP, DISTANCE_STEP)]

app = create_app()
with app.app_context():
    con = db.session.bind
    all_points_sql = Point.query.__str__()
    points_df = pd.read_sql(all_points_sql, con)
    logging.debug(points_df.head())

In [28]:
coords = points_df.as_matrix(columns=["point_lat", "point_long"])

for distance in distance_list:
    epsilon = distance / KMS_PER_RADIAN
    logging.debug("Epsilon for the current distance is equal {:.5f}".format(epsilon))
    db = DBSCAN(
        eps=epsilon, min_samples=3, algorithm="auto", metric="haversine"
    ).fit(np.radians(coords))
    cluster_labels = db.labels_

    all_lables = set(cluster_labels)
    all_lables.discard(-1)
    num_clusters = len(all_lables)

    clusters = [coords[cluster_labels == n] for n in range(num_clusters)]
    points_to_cluster = [points_df.point_id[cluster_labels == n] for n in range(num_clusters)]

    for num, cluster in enumerate(zip(clusters, points_to_cluster)):
        points_coords, points_id_list = cluster
        cluster_lat = MultiPoint(points_coords).centroid.x
        cluster_long = MultiPoint(points_coords).centroid.y
        print(cluster_lat, cluster_long)
        print(list(points_id_list))

/home/shevelsm/Yandex.Disk/myProjects/LearnPython/geo-treasure/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


44.64926446581199 37.99404374572651
[1, 2, 3, 7, 18, 19, 21, 29, 34, 36, 38, 40, 47, 51, 58, 59, 60, 70, 71, 85, 89, 94, 99, 109, 110, 111, 112, 125, 128, 130, 150, 155, 156, 164, 166, 167, 169, 174, 175, 176, 177, 179, 180, 185, 192, 194, 198, 200, 213, 221, 222, 230, 231, 232, 233, 235, 236, 244, 251, 255, 256, 258, 262, 284, 310, 311, 328, 359, 381, 388, 393, 466, 467, 468, 469, 470, 472, 475, 498, 507, 509, 510, 512, 516, 520, 544, 545, 561, 563, 574, 576, 579, 581, 588, 594, 601, 603, 608, 613, 614, 615, 630, 642, 647, 654, 655, 662, 663, 669, 691, 696, 703, 726, 728, 742, 743, 746, 748, 756, 762, 770, 782, 783, 788, 798, 804, 810, 815, 816, 823, 824, 825, 835, 837, 839, 845, 846, 847, 868, 869, 871, 872, 890, 892, 896, 915, 920, 921, 923, 925, 941, 942, 944, 952, 955, 959]
43.54125818947366 39.82659055438595
[4, 6, 10, 13, 16, 22, 25, 27, 35, 44, 53, 54, 66, 72, 76, 84, 95, 105, 114, 116, 117, 118, 119, 120, 127, 173, 178, 181, 214, 215, 238, 246, 260, 261, 264, 266, 267, 281, 28